In [ ]:
#
# A simulation of Hamming (15, 11) ECC
# https://en.wikipedia.org/wiki/Hamming_code
#
# https://afaan.dev
# (c) Afaan Bilal
#

In [1]:
from functools import reduce

# Pretty-print a 16-bit block
def pp(m, title = ''):
    print(title)
    for i in range(16):
        print(m[i], end=' ')
        if (i + 1) % 4 == 0:
            print()
    print()

# 11-bit message
message = [1,0,0,1,1,0,1,0,0,1,1]

# Reserved positions
reserved = [0, 1, 2, 4, 8]

# Create a 16-bit block
bits = [0] * 16

# Fill in the message
mi = 0
for i in range(16):
    if i in reserved:
        continue
    bits[i] = message[mi]
    mi += 1

# Message block
pp(bits, 'Message block:')

# Calculate parity bits
for i in range(1, 5):
    # The block associated with this parity bit
    block = [bits[j] for j in filter(lambda x: x & reserved[i], range(16))]
    # Calcuate and set the parity bit
    bits[reserved[i]] = reduce(lambda x, y: x ^ y, block)

# Calculate the 0-bit
bits[0] = reduce(lambda x, y: x ^ y, bits)

# The final block
pp(bits, 'Message with parity bits set:')


Message block:
0 0 0 1 
0 0 0 1 
0 1 0 1 
0 0 1 1 

Message with parity bits set:
1 1 1 1 
1 0 0 1 
0 1 0 1 
0 0 1 1 



In [2]:
# Reserved positions
reserved = [0, 1, 2, 4, 8]

# Fix the message (if bad)
def fix_message(bits, failed):
    # Can't fix if more than one bits are flipped
    if not failed[0]:
        print('Error: more than one bad bit. Cannot fix.')
        return

    # Find the bad bit
    bad_bit = 0
    for j in range(1, len(failed)):
        if failed[j]:
            bad_bit = bad_bit | j
    print('Bad bit: ' + str(bad_bit))

    # Flip the bad bit back to good value
    bits[bad_bit] = +(not bits[bad_bit])
    pp(bits, 'Fixed message:')

# Process, display and fix (if required) the message
def process_message(bits):
    # Passed parity checks
    failed = [False] * 9

    # Calculate parities
    for i in range(1, 5):
        # The block associated with this parity bit
        block = [bits[j] for j in filter(lambda x: x & reserved[i], range(16))]
        # Check parity
        failed[reserved[i]] = reduce(lambda x, y: x ^ y, block) != 0

    # Last parity check (to detect more than 1 flips)
    failed[0] = reduce(lambda x, y: x ^ y, bits) != 0

    print('Parity checks:')
    for i in range(5):
        print('Check ' + str(i) + ': ' + str(not failed[i]))
    if len([i for i in filter(lambda x: x, failed)]) == 0:
        print('*** Good message ***\n')
        pp(bits, 'Good message:')
    else:
        print('*** Bad message  ***\n')
        fix_message(bits, failed)


In [3]:
# Incoming bits (Good message)
bits = [1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1]

# Test incoming bits (Good message)
pp(bits, 'Incoming message:')
process_message(bits)

Incoming message:
1 1 1 1 
1 0 0 1 
0 1 0 1 
0 0 1 1 

Parity checks:
Check 0: True
Check 1: True
Check 2: True
Check 3: True
Check 4: True
*** Good message ***

Good message:
1 1 1 1 
1 0 0 1 
0 1 0 1 
0 0 1 1 



In [4]:
# Incoming bits (Bad message, 3rd-last bit flipped)
bits = [1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1]

# Test incoming bits (Bad message)
pp(bits, 'Incoming message:')
process_message(bits)


Incoming message:
1 1 1 1 
1 0 0 1 
0 1 0 1 
0 1 1 1 

Parity checks:
Check 0: False
Check 1: False
Check 2: True
Check 3: True
Check 4: False
*** Bad message  ***

Bad bit: 13
Fixed message:
1 1 1 1 
1 0 0 1 
0 1 0 1 
0 0 1 1 



In [5]:
# Incoming bits (Bad message, 2 bits flipped: 4th and 3rd-last bit flipped)
bits = [1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1]

# Test incoming bits (Bad message)
pp(bits, 'Incoming message:')
process_message(bits)

Incoming message:
1 1 1 0 
1 0 0 1 
0 1 0 1 
0 1 1 1 

Parity checks:
Check 0: True
Check 1: True
Check 2: False
Check 3: True
Check 4: False
*** Bad message  ***

Error: more than one bad bit. Cannot fix.
